In [ ]:
# get the pairs
# get the plate layout code
# what are the controls?
    # ask George what they used
    # SKP1-SKP2 / SKP1-BTRC?
    # A2, B2, D2, E2
# check if it's worth including the example ones

# requirements:
    # matched positives and negatives on the same plate
    # controls on the same plate


# NOTE: for future analysis, there is a TF_RRS pair in the isoform positives category 

In [34]:
import string
from collections import defaultdict

import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite

from ccsblib import paros_connection

In [2]:
qry = """
select distinct test_orf_ida, test_orf_idb, source 
from tf_validation.validation_configure a, 
     tf_validation.validation_source b,
     tf_validation.validation_nodes_1strnd c,
     tf_validation.validation_nodes_1strnd d
where a.standard_batch = 'TFv02' 
  and b.standard_batch = 'TFv02' 
  and a.orf_id1= b.orf_id1 
  and a.orf_id2 = b.orf_id2 
  and a.test_orf_ida = c.test_orf_id 
  and a.test_orf_idb = d.test_orf_id 
  and c.standard_batch = 'TFv02' 
  and d.standard_batch = 'TFv02'  
  and c.coverage_alt >= 0.95  
  and d.coverage_alt >=0.95 ;
"""
df = pd.read_sql(qry, paros_connection())
df = df.iloc[:, :-1].drop_duplicates()
"""
df = pd.concat([df, 
           pd.DataFrame(data=[(x, 0) for x in df['test_orf_ida'].unique()], 
                        columns=df.columns),
           pd.DataFrame(data=[(0, x) for x in df['test_orf_idb'].unique()], 
                        columns=df.columns)])
"""
df['test_orf_ida'] = 'F1_' + df['test_orf_ida'].astype(str)
df['test_orf_idb'] = 'F2_' + df['test_orf_idb'].astype(str)

In [3]:
df['test_orf_idb'].nunique()

411

In [4]:
df['test_orf_idb'].value_counts()

F2_100247    33
F2_100178    23
F2_100313    23
F2_100703    20
F2_100565    18
             ..
F2_71833      1
F2_3141       1
F2_14943      1
F2_4779       1
F2_101104     1
Name: test_orf_idb, Length: 411, dtype: int64

In [38]:
# this isn't working
# should also split F2 but somehow keep the isoforms together


# what if the network were gene-partner pairs and the size was calculated based on number of isoforms per gene?

'F1_10051'

In [66]:
# try splitting both and putting genes together at end
g = nx.Graph()
g.add_nodes_from(df['test_orf_ida'].unique(), bipartite=0)
g.add_nodes_from(df['test_orf_idb'].unique(), bipartite=1)
g.add_edges_from(df.loc[:, ['test_orf_ida', 'test_orf_idb']].values)


# should merge components with isoforms from the same gene
# get list of which edges from removed node are in each component
def size_including_controls(c):
    return len(c.edges) + len(c.nodes)

print('size of graph:', sum(size_including_controls(g.subgraph(x)) for x in nx.connected_components(g)))

max_size = 90  # TODO: 96 minus pre_assigned
n_edges_before = len(g.edges)
while True:
    components = list(sorted(nx.connected_components(g), key=len, reverse=True))
    lcc = g.subgraph(components[0])
    #print('size of lcc:', size_including_controls(lcc))  # DEBUG
    if size_including_controls(lcc) < max_size:
        break
    
    #nodes_to_split = nx.minimum_node_cut(lcc)
    #if len(nodes_to_split) == 0:
    #    raise UserWarning('minimum cut algorithm failed to find node(s)')

    nodes_to_split = set()
    lcc_with_nodes_removed = lcc.copy()
    while True:
        hub = sorted(lcc_with_nodes_removed.degree, key=lambda x: x[1], reverse=True)[0][0]
        nodes_to_split.add(hub)
        lcc_with_nodes_removed.remove_node(hub)
        components_after_removing_nodes = list(nx.connected_components(lcc_with_nodes_removed))
        if len(components_after_removing_nodes) > 1:
            break
    node_to_component = defaultdict(set)
    for i, new_component in enumerate(components_after_removing_nodes):
        for node in nodes_to_split:
            new_edges = [(a + f'_r{i + 1}', b) for a, b in g.edges(node) if b in new_component]
            if len(new_edges) > 0:
                g.add_edges_from(new_edges)
                node_to_component[node].add(i + 1)
    # deal with edges between the nodes to split
    for a, b in g.subgraph(nodes_to_split).edges:
        common_components = node_to_component[a].intersection(node_to_component[b])
        if len(common_components) > 0:
            i = list(common_components)[0]
            g.add_edges_from([(a + f'_r{i}', b + f'_r{i}')])
        else:
            g.add_edges_from([(a + f'_r{list(node_to_component[a])[0]}', b + f'_r{list(node_to_component[b])[0]}')])
    for node in nodes_to_split:
        g.remove_edges_from(list(g.edges(node)))
        g.remove_node(node)
if len(g.edges) != n_edges_before:
    raise UserWarning(f'{n_edges_before} edges before, {len(g.edges)} edges now. Should only affect number of nodes, not edges')

size of graph: 1747


In [67]:
sum(size_including_controls(g.subgraph(x)) for x in nx.connected_components(g))

1903

In [21]:
#TMP
import metis 


def size_including_controls(c):
    return len(c.edges) + len(c.nodes)

g = nx.Graph()
g.add_nodes_from(df['test_orf_ida'].unique(), bipartite=0)
g.add_nodes_from(df['test_orf_idb'].unique(), bipartite=1)
g.add_edges_from(df.loc[:, ['test_orf_ida', 'test_orf_idb']].values)
components = list(sorted(nx.connected_components(g), key=len, reverse=True))
lcc = g.subgraph(components[0])
print('size of lcc:', size_including_controls(lcc))  # DEBUG
"""
for nodes_to_cut in nx.all_node_cuts(lcc):
    print(len(nodes_to_cut))
    continue
    if len(nodes_to_cut) > 1:
        continue
    # size of second largest component
    component_with_nodes_removed = lcc.copy()
    for node in nodes_to_cut:
        component_with_nodes_removed.remove_node(node)
    new_components = list(nx.connected_components(component_with_nodes_removed))
    print(len(new_components[0]), len(new_components[1]))
    break
"""
__objective_value, partion_idx = metis.part_graph(lcc)
partion_idx = {v: i for i, v in zip(partion_idx, lcc.nodes)}

size of lcc: 665


In [23]:
# get list of edges between partitions
[(a, b) for a, b in lcc.edges if partion_idx[a] != partion_idx[b]]

[('F2_101369', 'F1_12624'),
 ('F2_101276', 'F1_100009829'),
 ('F2_100225', 'F1_7306'),
 ('F1_11965', 'F2_101278'),
 ('F2_100801', 'F1_4430'),
 ('F2_100801', 'F1_14023'),
 ('F2_100801', 'F1_6725'),
 ('F1_3741', 'F2_101278'),
 ('F2_101222', 'F1_1136'),
 ('F1_1136', 'F2_101221')]

In [68]:
from ortools.linear_solver import pywraplp

def optimize_plate(well_groups, max_size):
    plates = pack_bins([size_including_controls(x) for x in well_groups], max_size)
    #[well_groups]
    return plates


def pack_bins(weights, max_size):
    """
    copy pasted from https://developers.google.com/optimization/bin/bin_packing

    BUG: seems to give an empty bin at the end
    """
    data = {}
    data['weights'] = weights
    data['items'] = list(range(len(weights)))
    data['bins'] = data['items']
    data['bin_capacity'] = max_size
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    # Variables
    # x[i, j] = 1 if item i is packed in bin j.
    x = {}
    for i in data['items']:
        for j in data['bins']:
            x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))
    # y[j] = 1 if bin j is used.
    y = {}
    for j in data['bins']:
        y[j] = solver.IntVar(0, 1, 'y[%i]' % j)
    # Constraints
    # Each item must be in exactly one bin.
    for i in data['items']:
        solver.Add(sum(x[i, j] for j in data['bins']) == 1)
    # The amount packed in each bin cannot exceed its capacity.
    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['weights'][i] for i in data['items']) <= y[j] *
            data['bin_capacity'])
    # Objective: minimize the number of bins used.
    solver.Minimize(solver.Sum([y[j] for j in data['bins']]))
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        num_bins = 0.
        bins = []
        for j in data['bins']:
            if y[j].solution_value() == 1:
                bin_items = []
                bin_weight = 0
                for i in data['items']:
                    if x[i, j].solution_value() > 0:
                        bin_items.append(i)
                        bin_weight += data['weights'][i]
                if bin_weight > 0:
                    num_bins += 1
                    bins.append(bin_items)
        return bins
    else:
        print('The problem does not have an optimal solution.') 

plates = optimize_plate(
                        well_groups=list(g.subgraph(x) for x in nx.connected_components(g)),
                        max_size=90
)

In [69]:
well_groups = list(g.subgraph(x) for x in nx.connected_components(g))
pairs = [[edge for i in plate for edge in well_groups[i].edges] for plate in plates]
# NOTE: need to sort F1 and F2
pairs = [[tuple(sorted(pair)) for pair in pairs_on_a_plate] for pairs_on_a_plate in pairs]
pairs = [[(int(f1.split('_')[1]), int(f2.split('_')[1])) for f1, f2 in pairs_on_a_plate] for pairs_on_a_plate in pairs]


def add_controls(pairs_on_plate):
    wells = []
    for x, y in pairs_on_plate:
        wells.append((x, y))
        if (x, 0) not in wells:
            wells.append((x, 0))
        if (0, y) not in wells:
            wells.append((0, y))
    return wells


items_per_plate = [add_controls(plate_pairs) for plate_pairs in pairs]
max_size = 90  # change to plate size minus pre assigned wells
if any(len(x) > max_size for x in items_per_plate):
    raise UserWarning('Too many items to fit on a plate')

pre_assigned_wells = {'B02': (6671, 4656),  # SKP1 SKP2
                      'C02': (6671, 4656),  # SKP1 SKP2
                      'D02': (6671, 9477),  # SKP1 BTRC
                      'E02': (6671, 9477),  # SKP1 BTRC
                      'G12': (np.nan, np.nan),
                      'H12': (np.nan, np.nan)}

well_codes_96 = [l + str(i).zfill(2) for l in string.ascii_uppercase[:8] for i in range(1, 13)]
plate_layout = []
for plate_idx, items in enumerate(items_per_plate):
    item_idx = 0
    for well in well_codes_96:
        if well in pre_assigned_wells:
            plate_layout.append((plate_idx + 1, well, *pre_assigned_wells[well]))
        elif item_idx < len(items):
            plate_layout.append((plate_idx + 1, well, *items[item_idx]))
            item_idx += 1
        else:
            plate_layout.append((plate_idx + 1, well, np.nan, np.nan))

# add ORF ids for SKP1/2 and the other
out = pd.DataFrame(data=plate_layout, 
                   columns=['plate_number', 'well', 'test_orf_ida', 'test_orf_idb'])
# change columns to int with missing values since they're converted to float by default
out['test_orf_ida'] = out['test_orf_ida'].astype('Int64')
out['test_orf_idb'] = out['test_orf_idb'].astype('Int64')

In [72]:
# run a bunch of checks on the final layout
n_duplicate_pairs =  (out.loc[
            out['test_orf_ida'].notnull()
            & out['test_orf_idb'].notnull()
            & (out['test_orf_ida'] != 0)
            & (out['test_orf_idb'] != 0)
            & ~out['well'].isin(pre_assigned_wells.keys()),
            ['test_orf_ida', 'test_orf_idb']]
            .duplicated()
            .sum())
if n_duplicate_pairs > 0:
    raise UserWarning('Unexpected duplicate pairs')
# no duplicates on the same plate except controls
if out.loc[~out['well'].isin(pre_assigned_wells.keys()) & 
        (out['test_orf_ida'].notnull()
            | out['test_orf_idb'].notnull()),
       ['plate_number', 'test_orf_ida', 'test_orf_idb']].duplicated().sum() > 0:
       raise UserWarning('Unexpected duplicates')

# check exactly one control is on each plate for each ORF in a pair
EMPTY_F1_ORF_ID = 0
EMPTY_F2_ORF_ID = 0
for plate_number in out['plate_number'].unique():
    for orf_ida in out.loc[(out['plate_number'] == plate_number) & 
                           ~out['well'].isin(pre_assigned_wells.keys()) &
                           (out['test_orf_ida'] != EMPTY_F1_ORF_ID) &
                           out['test_orf_ida'].notnull() &
                           (out['test_orf_idb'] != EMPTY_F2_ORF_ID),
                            'test_orf_ida'].unique():
        n_control = (((out['plate_number'] == plate_number) &
                      (out['test_orf_ida'] == orf_ida) &
                      (out['test_orf_idb'] == EMPTY_F2_ORF_ID))
                     .sum())
        if n_control != 1:
            print(f'WARNING: {n_control} empty-F2 controls for {orf_ida} on plate {plate_number}')
    for orf_idb in out.loc[(out['plate_number'] == plate_number) & 
                           ~out['well'].isin(pre_assigned_wells.keys()) &
                           (out['test_orf_idb'] != EMPTY_F2_ORF_ID) &
                           out['test_orf_idb'].notnull() &
                           (out['test_orf_ida'] != EMPTY_F1_ORF_ID), 
                           'test_orf_idb'].unique():
        n_control = (((out['plate_number'] == plate_number) &
                      (out['test_orf_idb'] == orf_idb) &
                      (out['test_orf_ida'] == EMPTY_F1_ORF_ID))
                     .sum())
        if n_control != 1:
            print(f'WARNING: {n_control} empty-F1 controls for {orf_ida} on plate {plate_number}')

PLATE_N_WELLS = 96
if not out['well'].isin(well_codes_96).all():
    raise UserWarning('Invalid well code')
if out.duplicated(subset=['plate_number', 'well']).any():
    raise UserWarning('Unexpected well duplicates')
if (out.groupby('plate_number').size() != PLATE_N_WELLS).any():
    raise UserWarning(f'Plate without {PLATE_N_WELLS} wells')

df_in = df.copy()
df_in = df_in.applymap(lambda x: int(x.split('_')[1]))
# check all the pairs are in the output
pairs_in = [tuple(pair) for pair in df_in.values]
pairs_out = set([tuple(pair) for pair in out.loc[:, ['test_orf_ida', 'test_orf_idb']].values])
if not all([pair in pairs_out for pair in pairs_in]):
    print([pair for pair in pairs_in if pair not in pairs_out])
    raise UserWarning('Missing some pairs')


# isoforms of the same gene with a speicific partner are on the same plate 
qry = """
select *
from tf_isoforms.tfiso4100_orfid_symbol_map;
"""
iso_orf_id_to_gene = pd.read_sql(qry, paros_connection()).set_index('orfid')['symbol'].to_dict()
tmp = out.copy()
tmp['tf_gene'] = tmp['test_orf_idb'].map(iso_orf_id_to_gene)
gene_level_pairs = tmp.loc[tmp['tf_gene'].notnull() & 
                           (tmp['test_orf_ida'] != EMPTY_F1_ORF_ID), 
                           ['plate_number', 'test_orf_ida', 'tf_gene']]
print(gene_level_pairs.shape[0], 'isoform level pairs')
print(gene_level_pairs.drop_duplicates().shape[0], 'gene level pairs')
n_plate_per_gene_level_pair = (gene_level_pairs.groupby(['tf_gene', 'test_orf_ida'])
                                            ['plate_number']
                                            .nunique())
if (n_plate_per_gene_level_pair > 1).any():
    print(n_plate_per_gene_level_pair[n_plate_per_gene_level_pair > 1])
    raise UserWarning('Different isoforms of the same gene with the same partner, split across multiple plates')

344 isoform level pairs
239 gene level pairs


In [81]:
# print some details
print((out['test_orf_ida'] == 0).sum(), 'empty F1')
print((out['test_orf_idb'] == 0).sum(), 'empty F2')
print()
print('empty wells per plate')
print((out['test_orf_ida'].isnull() & out['test_orf_idb'].isnull()).groupby(out['plate_number']).sum())


480 empty F1
554 empty F2

empty wells per plate
plate_number
1      7.0
2      5.0
3     12.0
4      4.0
5      6.0
6      5.0
7     13.0
8     21.0
9     11.0
10    15.0
11    17.0
12     4.0
13     3.0
14     3.0
15     4.0
16     2.0
17     3.0
18     4.0
19     2.0
20     2.0
21     3.0
22    59.0
dtype: float64


In [73]:
out.to_csv('../output/mN2H_plate_layout_2022-05-06.tsv', sep='\t', index=False)

In [14]:
[n for n in g.nodes if n.startswith('F1_100009829')]

['F1_100009829',
 'F1_100009829_r1',
 'F1_100009829_r1_r1',
 'F1_100009829_r1_r1_r1',
 'F1_100009829_r1_r1_r3']

In [20]:
[n for n in g.nodes if n.startswith('F2_100178')]

['F2_100178',
 'F2_100178_r1',
 'F2_100178_r2',
 'F2_100178_r3',
 'F2_100178_r4',
 'F2_100178_r5',
 'F2_100178_r6',
 'F2_100178_r7',
 'F2_100178_r8',
 'F2_100178_r1_r1',
 'F2_100178_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r4',
 'F2_100178_r1_r1_r1_r5',
 'F2_100178_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r2',
 'F2_100178_r1_r1_r1_r1_r3',
 'F2_100178_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r1_r2',
 'F2_100178_r1_r1_r1_r1_r1_r3',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1_r2',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1',
 'F2_100178_r1_r1_r1_r1

In [21]:
g.edges('F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r2')

EdgeDataView([('F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r2', 'F1_1136')])

In [22]:
g.edges('F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1')

EdgeDataView([('F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1', 'F1_13474'), ('F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1', 'F1_13577'), ('F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1', 'F1_6636'), ('F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1', 'F1_53886'), ('F2_100178_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1', 'F1_172')])

In [18]:
g.edges('F1_100009829_r1_r1_r1')

EdgeDataView([('F1_100009829_r1_r1_r1', 'F2_100455_r1_r1_r1_r1'), ('F1_100009829_r1_r1_r1', 'F2_100565_r1_r1_r1_r1_r1_r1_r1'), ('F1_100009829_r1_r1_r1', 'F2_100313_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1'), ('F1_100009829_r1_r1_r1', 'F2_100703_r1_r1_r1_r1_r1_r1_r1_r1_r1_r1')])

In [19]:
g.edges('F1_100009829_r1_r1_r3')

EdgeDataView([('F1_100009829_r1_r1_r3', 'F2_101399'), ('F1_100009829_r1_r1_r3', 'F2_101276'), ('F1_100009829_r1_r1_r3', 'F2_101278')])

In [158]:
df.loc[df['test_orf_ida'] == 'F1_100009829']

,test_orf_ida,test_orf_idb
48,F1_100009829,F2_101399
49,F1_100009829,F2_101276
50,F1_100009829,F2_100313
51,F1_100009829,F2_100178
52,F1_100009829,F2_100247
53,F1_100009829,F2_100703
54,F1_100009829,F2_101278
55,F1_100009829,F2_100455
56,F1_100009829,F2_100565


In [162]:
out.loc[(out['test_orf_ida'] == 100009829) & (out['test_orf_idb'] == 100178)]

,plate_number,well,test_orf_ida,test_orf_idb


In [164]:
out.loc[(out['test_orf_idb'] == 100178)]

,plate_number,well,test_orf_ida,test_orf_idb
149,2,E06,2595,100178
150,2,E07,0,100178
159,2,F04,13474,100178
167,2,F12,53326,100178
170,2,G03,3192,100178
264,3,G01,53726,100178
265,3,G02,0,100178
270,3,G07,11179,100178
393,5,A10,13577,100178
394,5,A11,0,100178


In [154]:
pairs_out

{(3818, 0),
 (2427, 0),
 (0, 101254),
 (53279, 100114),
 (100014695, 11695),
 (3120, 0),
 (8313, 3141),
 (100066358, 0),
 (1942, 5742),
 (14864, 101468),
 (54731, 101414),
 (8141, 100118),
 (0, 5868),
 (1942, 0),
 (0, 100063),
 (0, 2341),
 (11965, 0),
 (10016, 101070),
 (100009829, 100455),
 (12090, 0),
 (9556, 56762),
 (10650, 100044),
 (0, 100593),
 (8313, 0),
 (8141, 100468),
 (14087, 100091),
 (0, 3774),
 (12169, 0),
 (5231, 101222),
 (7076, 100409),
 (3750, 0),
 (53726, 100178),
 (1564, 100340),
 (10051, 101414),
 (8264, 0),
 (14023, 100048),
 (14210, 11786),
 (6003, 0),
 (0, 55982),
 (919, 0),
 (7735, 0),
 (130, 101174),
 (56482, 101265),
 (70764, 1102),
 (8113, 100684),
 (0, 4955),
 (9318, 3180),
 (0, 13177),
 (0, 100282),
 (3385, 0),
 (0, 11636),
 (0, 14754),
 (3425, 101407),
 (11755, 0),
 (5597, 100165),
 (0, 101221),
 (3425, 100769),
 (5057, 101469),
 (3567, 100178),
 (10051, 100645),
 (3245, 11747),
 (100015383, 100069009),
 (12007, 100157),
 (11968, 0),
 (11706, 0),
 (0, 10

In [57]:
from itertools import cycle
# find hub
# get all pairs and control
# get all other pairs and their controls


# will have to add exclusion list
# biggest hub: orfb: 100247
def expand_pairs(starting_orf_id, start_with_orf_a, exclude=False):
    """
    """
    order = ['test_orf_ida', 'test_orf_idb']
    if start_with_orf_a:
        order = list(reversed(order))
    pairs = (df[order[1]] == starting_orf_id)
    for node_column in cycle(order):
        matched = (df[node_column].isin(df.loc[pairs, node_column].unique())
                & (df[node_column] != 0)
                & ~pairs
                & ~exclude)
        if matched.sum() == 0:
            break
        pairs = pairs | matched
    return pairs


expand_pairs(starting_orf_id=100247,
             start_with_orf_a=False).sum()

665

In [68]:
# get the individual components
# maybe remove the controls to start
# figure out the max size of 96 - n controls = number edges + number nodes
# 1. take the largest component
# 2. split the largest non-isoform side hub into two, in a way that 
# minimizes the size of larger of the two components
# 3. are all components less than 96 - n controls? if not go to 1.

# if a bunch of components are > half the size of the plate then
# this could be an issue. Need another step to the algorithm?
df.loc[df['test_orf_idb'] == 100247, :].iloc[:15, :]

,test_orf_ida,test_orf_idb
52,100009829,100247
59,2595,100247
61,120,100247
68,13474,100247
73,13577,100247
76,12627,100247
80,53326,100247
85,6636,100247
89,3192,100247
94,53726,100247


In [34]:
(df['test_orf_ida'].nunique(), df['test_orf_idb'].nunique())

(552, 412)

In [ ]:
import string


class Plate:
    def __init__(self, n_rows, n_columns, empty_name='empty'):
        self.n_rows = n_rows
        self.n_columns = n_columns
        self.empty_name = empty_name
        self.grid = [[empty_name for i in range(n_columns)] for j in range(n_rows)]

    def add_matrix(self, matrix, pos_top_left, transpose=False):
        n_rows = len(matrix)
        n_columns = len(matrix[0])
        if n_rows + pos_top_left[0] > self.n_rows or n_columns + pos_top_left[1] > self.n_columns:
            raise ValueError('Does not fit: {} rows {} columns, top left position: {} \n{}'.format(n_rows,
                                                                  n_columns,
                                                                  pos_top_left,
                                                                  self))
        for i, row in enumerate(matrix):
            row_index = i + pos_top_left[0]
            for j, well in enumerate(row):
                column_index = j + pos_top_left[1]
                if self.grid[row_index][column_index] != self.empty_name:
                    raise ValueError('Well already occupied: {} {}\n{}'.format(row_index, column_index, self))
                self.grid[row_index][column_index] = well

    def lowest_unoccupied_row(self):
        for i in range(self.n_rows):
            if self.grid[i] == [self.empty_name] * self.n_columns:
                return i
        return None

    def leftmost_unoccupied_column(self):
        for i in range(self.n_columns):
            if [row[i] for row in self.grid] == [self.empty_name] * self.n_rows:
                return i
        return None

    def is_empty(self):
        return all(all(x == self.empty_name for x in row) for row in self.grid)

    def is_full(self):
        return all(all(x != self.empty_name for x in row) for row in self.grid)

    def row_is_empty(self, row_index):
        return all(x == self.empty_name for x in self.grid[row_index])

    def column_is_empty(self, column_index):
        return all(row[column_index] == self.empty_name for row in self.grid)

    def empty_wells(self):
        return {string.ascii_uppercase[i] + str(j + 1).zfill(2) for i, row in enumerate(self.grid) 
                                                                for j, x in enumerate(row) 
                                                                if x == self.empty_name}

    def add_empty_row_below(self):
        self.grid.append([self.empty_name] * self.n_columns)
        self.n_rows += 1

    def empty_rectangles_from_bottom_right(self):
        """Starting from the bottom right corner, all different rectangles of empty space,
        which are not contained within another. Assumes plate is filled from top left.
        
            Returns [(int, int)]: number of rows, number of columns
        
        """
        if self.is_empty():
            return [(self.n_rows, self.n_columns)]
        if self.is_full():
            return []

        def count_empty_cells_from_right(row):
            for i in range(1, self.n_columns + 1):
                if row[-i] != self.empty_name:
                    return i - 1
            else:
                return self.n_columns

        rectangles = []
        n_columns = count_empty_cells_from_right(self.grid[-1])  #sum(x == self.empty_name for x in self.grid[-1])
        for row_index in reversed(range(self.n_rows)):
            n_empty_cells = count_empty_cells_from_right(self.grid[row_index])  #sum(x == self.empty_name for x in self.grid[row_index])
            if n_empty_cells == 0:
                break
            if n_empty_cells < n_columns:
                rectangles.insert(0, ((self.n_rows - row_index) - 1,  n_columns))
                n_columns = n_empty_cells
        else:
            rectangles.insert(0, (self.n_rows,  n_columns))
        return rectangles

    def xo_picture(self):
        print('\n'.join('|'.join('O' if cell == self.empty_name else 'X' for cell in row) for row in self.grid))

    def __repr__(self):
        return '\n'.join('|'.join(row) for row in self.grid)